In [25]:
import pandas as pd

df = pd.read_csv('data.csv')

In [26]:
df.head()

,match_id,radiant_adv_nw,radiant_adv_xp,radiant_adv_lh,radiant_win
0,0,707,1238,-39,True
1,1,-193,-1224,9,False
2,2,1035,1470,36,False
3,3,2464,2822,-7,False
4,4,6967,5261,150,True


In [27]:
X = df.iloc[:,1:4]
y = df['radiant_win'].astype(int).values

In [28]:
from sklearn.model_selection import train_test_split# implementing train-test-split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

In [38]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000, random_state=0)

clf.fit(X_train, y_train) # Training

RandomForestClassifier(n_estimators=1000, random_state=0)

In [39]:
# Apply the Classifier we trained to the test data (which, remember, it has never seen before)
y_pred = clf.predict(X_test)

In [40]:
# View the predicted probabilities of the first 10 observations
clf.predict_proba(X_test)[0:10]

array([[0.555, 0.445],
       [0.147, 0.853],
       [0.73 , 0.27 ],
       [0.21 , 0.79 ],
       [0.388, 0.612],
       [0.834, 0.166],
       [0.083, 0.917],
       [0.278, 0.722],
       [0.567, 0.433],
       [0.953, 0.047]])

In [41]:
### Evaluate classifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test,y_pred))

[[3741 2203]
 [1810 4394]]
              precision    recall  f1-score   support

           0       0.67      0.63      0.65      5944
           1       0.67      0.71      0.69      6204

    accuracy                           0.67     12148
   macro avg       0.67      0.67      0.67     12148
weighted avg       0.67      0.67      0.67     12148

0.6696575567994731
